<a href="https://colab.research.google.com/github/saksham1211/Marketingai_Internship/blob/master/Topic_Modeling_for_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install pyLDAvis

     |████████████████████████████████| 1.6MB 2.8MB/s 
     |████████████████████████████████| 552kB 16.9MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=c7b95ca0614431fb4a7203b291e0ef5488437bbedb6477fad5edfdde4d83f1d0
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=3bc8bec175867f63fde4425cc8df1603490ee6e2f642400b1d98bb640d1896bb
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [0]:
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import random
import re, nltk, spacy, gensim
import pyLDAvis
import pyLDAvis.sklearn
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns', 50)

In [0]:
df = pd.read_csv('/content/Book1.csv', encoding="latin-1")
df.set_index('CATEGORY', inplace = True)
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['DESCRIPTION'].values.astype('U'))
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [0]:
indices = pd.Series(df.index)
def recommendations(name, cosine_similarities = cosine_similarities):
    
    recommended_category = []
    
   
    idx = indices[indices == name].index[0]

    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending = False)

    top_10_indexes = list(score_series.iloc[1:6].index)

    for i in top_10_indexes:
        recommended_category.append(list(df.index)[i])
        
    return recommended_category

In [0]:
recommendations('Schwab_IRA')

['Schwab_IRA', 'Schwab_IRA', 'Schwab_IRA', 'Schwab_IRA', 'Schwab_IRA']

In [0]:
df.loc['Schwab_IRA'].DESCRIPTION

CATEGORY
Schwab_IRA    Distributions from your IRA are generally subj...
Schwab_IRA    Your tax withholding election will remain in e...
Schwab_IRA    Withholding income taxes may subject you to IR...
Schwab_IRA    Please select a withholding instruction for th...
Schwab_IRA    State tax withholding may be required from you...
Schwab_IRA    While Schwab makes every effort to obtain info...
Schwab_IRA    A Schwab Qualified Plan is a Schwab Profit Sha...
Schwab_IRA    The Cash Features Program is a service that of...
Schwab_IRA    You understand that additional information abo...
Schwab_IRA    The Bank Sweep feature is designated as the Ca...
Schwab_IRA    Your deposits at each Sweep Bank are insured b...
Schwab_IRA    You understand and agree that Schwab may (1) m...
Schwab_IRA    Schwab will notify you in writing of changes t...
Schwab_IRA    You will receive account statements, trade con...
Schwab_IRA    For complete information, please see Important...
Schwab_IRA    If you already ha

In [0]:
df.loc["Ameritrade Institutional"].DESCRIPTION

CATEGORY
Ameritrade Institutional    A. Transfer from an ACAT eligible Brokerage Fi...
Ameritrade Institutional    When transferring from a joint account at a co...
Ameritrade Institutional    If this account is a qualified retirement acco...
Ameritrade Institutional    Unless otherwise indicated in the instructions...
Ameritrade Institutional    Unless otherwise indicated in the instructions...
Ameritrade Institutional    Instructions to transfer securities, mutual fu...
Ameritrade Institutional    Section 1 â TD Ameritrade Institutional Acco...
Ameritrade Institutional    Section 2 â Delivering Firm Information â ...
Ameritrade Institutional    Section 3 â Transfer details; see below for ...
Ameritrade Institutional    A. Transfer from an ACAT eligible Brokerage Fi...
Ameritrade Institutional    Section 3A â Brokerage Firm Transfer  â¢  F...
Ameritrade Institutional    Brokerage Firm, Bank, Insurance/Annuity Co., T...
Ameritrade Institutional    To transfer the proceeds of

In [0]:
df.loc["Merrilllynch_CRA"].DESCRIPTION

CATEGORY
Merrilllynch_CRA    Use this form to transfer all or a portion of ...
Merrilllynch_CRA    For best results, please complete the informat...
Merrilllynch_CRA    Many firms transfer assets using an automated ...
Merrilllynch_CRA    Remember to update any automatic deposits or w...
Merrilllynch_CRA    Some transfers may result in tax consequences,...
Merrilllynch_CRA    If youâre rolling over assets from a 401(k) ...
Merrilllynch_CRA    Before indicating which assets to transfer, yo...
Merrilllynch_CRA    If you are transferring mutual funds directly ...
Merrilllynch_CRA    Most financial institutions require either wri...
Merrilllynch_CRA    Use the sections below to list the assets you ...
Merrilllynch_CRA    Use the sections below to list the assets you ...
Merrilllynch_CRA    Â£Â£ Immediately liquidate CDs and transfer in...
Merrilllynch_CRA    Securities include stocks, bonds, exchange tra...
Merrilllynch_CRA    Transfer your account to Merrill Lynch as you ...
Merrilllync

In [0]:
df.reset_index(inplace=True)

In [0]:
df = pd.concat([pd.Series(str(row['CATEGORY']), str(row['DESCRIPTION']).split('. '))              
                    for _, row in df.iterrows()]).reset_index()
df.columns = ['sentence', 'CATEGORY']
df['sentence'] = df['sentence'].map(lambda x: re.sub(r'\W+', ' ', x))
print('We have ', len(df), 'sentences in total')

We have  207 sentences in total


In [0]:
df.loc[df['CATEGORY'] == 'Merrilllynch_CRA']

,sentence,CATEGORY
0,Use this form to transfer all or a portion of ...,Merrilllynch_CRA
1,For best results please complete the informati...,Merrilllynch_CRA
2,If you need more room for information or signa...,Merrilllynch_CRA
3,Use one form for each account that youâ re tra...,Merrilllynch_CRA
4,Many firms transfer assets using an automated ...,Merrilllynch_CRA
...,...,...
69,MLPF S is a registered broker dealer Member SI...,Merrilllynch_CRA
70,Next with the expertise of our Chief Investmen...,Merrilllynch_CRA
71,Wealth management is an ongoing process,Merrilllynch_CRA
72,You ll meet with your advisor often and have i...,Merrilllynch_CRA


In [0]:
a = 0
for i in range(a,a+8):
    print(df.sentence[i])
    print()

Use this form to transfer all or a portion of your account from another financial institution to your Merrill Lynch Pierce Fenner Smith Incorporated â Merrill Lynchâ account

For best results please complete the information requested below in parts 1 to 4 using ALL CAPITAL letters and black ink

If you need more room for information or signatures use a copy of the relevant page or a blank sheet

Use one form for each account that youâ re transferring

Many firms transfer assets using an automated transfer process which typically takes 5 to 6 business days

Other firms either donâ t use the automated process or must process some assets manually which takes more time

The following estimated time frames for manual transfers are from the date of receipt of instructions by the delivering firm and may vary greatly

Remember to update any automatic deposits or withdrawals youâ ve set up on the account youâ re transferring to Merrill Lynch



In [0]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=3,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             max_features=3000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(df['sentence'])

In [0]:
lda_model = LatentDirichletAllocation(n_components=40, # Number of topics
                                      learning_method='online',
                                      random_state=0,       
                                      n_jobs = -1  # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=40, n_jobs=-1,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)


In [0]:
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
24     -35.071960  120.913330       1        1  10.998056
8      -10.236718  -98.752800       2        1  10.575023
36      32.493645  -95.627762       3        1   9.232797
16     -60.065056  -97.236763       4        1   9.203152
18    -109.824615  -37.864731       5        1   6.194255
26      57.374569   81.427208       6        1   4.604746
17     -78.958359  -54.998604       7        1   4.412867
29     -46.934189   79.972397       8        1   3.746804
4      -82.505127   63.441189       9        1   3.148856
34      68.536163  -20.609587      10        1   3.118362
5       74.643387  -61.240322      11        1   2.920819
28      79.415161   43.600166      12        1   2.665610
32    -110.979904   29.144428      13        1   2.609834
6      -89.607536   -0.271766      14        1   2.566266
11      12.958403  103.941628      15        1   2.363268
0       95.245552    5.248045      16        1   2.239125
21     -77.273407   32.569561      17        1   2.097652
12      22.535919   68.256081      18        1   2.065717
35      54.558868   11.016172      19        1   1.563426
38     -11.520891   71.692360      20        1   1.411572
13      13.407472  -32.251205      21        1   1.186982
15      39.372089  -46.780273      22        1   0.902568
2      -44.591724   43.353535      23        1   0.883336
37      43.640213   39.840588      24        1   0.833878
30     -38.559128  -31.018372      25        1   0.826832
23     -35.729336  -62.561195      26        1   0.758145
20     -63.467197  -21.385942      27        1   0.697272
7       33.166389  -12.453777      28        1   0.675333
22      13.396214   39.931297      29        1   0.675333
3        8.036204  -61.665207      30        1   0.675333
25     -14.053920  -43.894989      31        1   0.599677
33     -55.282352    9.175754      32        1   0.547421
27     -14.825848   42.628105      33        1   0.547421
19     -30.839928   21.569729      34        1   0.484281
39     -14.442764    4.544766      35        1   0.422230
9       -0.548140   20.846491      36        1   0.362357
31      23.449495   13.431845      37        1   0.357593
10       7.529176   -5.451264      38        1   0.275268
14     -34.475109   -5.964467      39        1   0.275268
1      -12.165504  -18.795368      40        1   0.275268, topic_info=            Term       Freq      Total Category  logprob  loglift
200     transfer  31.000000  31.000000  Default  30.0000  30.0000
35          cash  29.000000  29.000000  Default  29.0000  29.0000
215  withholding  16.000000  16.000000  Default  28.0000  28.0000
194          tax  24.000000  24.000000  Default  27.0000  27.0000
18        assets  23.000000  23.000000  Default  26.0000  26.0000
..           ...        ...        ...      ...      ...      ...
3        account   0.027152  59.998165  Topic40  -5.0448  -1.8054
193        sweep   0.021009   9.910215  Topic40  -5.3013  -0.2612
126      merrill   0.021824  13.957920  Topic40  -5.2632  -0.5656
72      election   0.019368   6.431239  Topic40  -5.3826   0.0899
57    delivering   0.019564   9.004792  Topic40  -5.3725  -0.2366

[1853 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
0         3  0.184597          401
0         5  0.184597          401
0         9  0.184597          401
0        21  0.184597          401
1        11  0.280888         acat
...     ...       ...          ...
215       8  0.059173  withholding
215      16  0.059173  withholding
215      20  0.059173  withholding
215      24  0.059173  withholding
216      12  0.490554      writing

[660 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[25, 9, 37, 17, 19, 27, 18, 30, 5, 35, 6, 29, 33, 7, 12, 1, 22, 13, 36, 39, 14, 16, 3, 38, 31, 24, 21, 8, 23, 4, 26, 34, 28, 20, 40, 

In [0]:
# Show top 20 keywords for each topic
import numpy as np

def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19
Topic 0,goals,help,document,change,ira,distributions,financial,needs,tax,withholding,kind,election,revoke,account,provide,approach,copy,certificate,offer,products
Topic 1,transfer,liquidation,kind,assets,account,complete,outstanding,section,fees,balance,depending,transferring,deposits,lynch,set,merrill,designation,credit,sweep,requests
Topic 2,form,transfer,use,plan,accounts,box,shares,check,ameritrade,firms,brokerage,change,distribution,available,held,number,writing,free,access,annuity
Topic 3,market,financial,decisions,makes,understand,make,best,funds,request,result,penalties,transfers,assets,mutual,lynch,merrill,proprietary,tax,certain,transferable
Topic 4,merrill,lynch,account,custodian,products,form,transfer,incorporated,trading,trade,deposit,access,sponsored,online,free,fees,ameritrade,successor,retirement,financial
Topic 5,transfer,annuity,insurance,trust,check,agent,box,firm,brokerage,bank,acat,partial,money,etfs,new,401,authorized,fund,ira,section
Topic 6,assets,lynch,merrill,list,want,liquidate,use,request,document,bank,guarantee,institutions,deposits,certificate,financial,delivering,including,account,transfer,transferred
Topic 7,transfers,instructions,time,firm,date,delivering,following,section,account,number,applicable,complete,sign,trading,list,financial,type,partial,shares,sent
Topic 8,account,bank,transferring,deposit,schwab,sweep,distribution,form,certificate,deposits,section,complete,paperless,accounts,party,brokerage,minimum,insurance,documents,trade
Topic 9,successor,custodian,deposit,certificate,bank,transferring,holds,section,complete,incorporated,account,features,losing,applicable,use,election,indicated,designated,amended,limited


In [0]:
# Create Document - Topic Matrix
lda_output = lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(40)]

# index names
docnames = ["Doc" + str(i) for i in range(0,207)]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

In [0]:
df_document_topic.head(10)

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,Topic20,Topic21,Topic22,Topic23,Topic24,Topic25,Topic26,Topic27,Topic28,Topic29,Topic30,Topic31,Topic32,Topic33,Topic34,Topic35,Topic36,Topic37,Topic38,Topic39,dominant_topic
Doc0,0.00,0.00,0.00,0.00,0.92,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4
Doc1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.18,0.00,0.00,0.00,0.13,0.00,0.00,0.00,0.32,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.00,0.00,0.00,0.00,16
Doc2,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.76,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,16
Doc3,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.80,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,8
Doc4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.89,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,28
Doc5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.86,0.00,38
Doc6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.88,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,7
Doc7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.41,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.32,0.00,0.00,0.00,0.14,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8
Doc8,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.86,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,29
Doc9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.86,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,29


In [0]:
df_document_topic.reset_index(inplace=True)
df_sent_topic= pd.merge(df, df_document_topic, left_index=True, right_index=True)
df_sent_topic.drop('index', axis=1, inplace=True)

In [0]:
df_sent_topic.head(10)

,sentence,CATEGORY,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,Topic20,Topic21,Topic22,Topic23,Topic24,Topic25,Topic26,Topic27,Topic28,Topic29,Topic30,Topic31,Topic32,Topic33,Topic34,Topic35,Topic36,Topic37,Topic38,Topic39,dominant_topic
0,Use this form to transfer all or a portion of ...,Merrilllynch_CRA,0.00,0.00,0.00,0.00,0.92,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4
1,For best results please complete the informati...,Merrilllynch_CRA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.18,0.00,0.00,0.00,0.13,0.00,0.00,0.00,0.32,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.00,0.00,0.00,0.00,16
2,If you need more room for information or signa...,Merrilllynch_CRA,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.76,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,16
3,Use one form for each account that youâ re tra...,Merrilllynch_CRA,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.80,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,8
4,Many firms transfer assets using an automated ...,Merrilllynch_CRA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.89,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,28
5,Other firms either donâ t use the automated pr...,Merrilllynch_CRA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.86,0.00,38
6,The following estimated time frames for manual...,Merrilllynch_CRA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.88,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,7
7,Remember to update any automatic deposits or w...,Merrilllynch_CRA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.41,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.32,0.00,0.00,0.00,0.14,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8
8,Some transfers may result in tax consequences ...,Merrilllynch_CRA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.86,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,29
9,Consult your tax and financial professionals b...,Merrilllynch_CRA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.86,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,29


In [0]:
df_sent_topic.loc[df_sent_topic['CATEGORY'] == 'Merrilllynch_CRA'][['sentence', 'dominant_topic']]

,sentence,dominant_topic
0,Use this form to transfer all or a portion of ...,4
1,For best results please complete the informati...,16
2,If you need more room for information or signa...,16
3,Use one form for each account that youâ re tra...,8
4,Many firms transfer assets using an automated ...,28
...,...,...
69,MLPF S is a registered broker dealer Member SI...,16
70,Next with the expertise of our Chief Investmen...,16
71,Wealth management is an ongoing process,12
72,You ll meet with your advisor often and have i...,16


In [0]:
df_sent_topic.loc[df_sent_topic['dominant_topic'] == 4][['sentence', 'dominant_topic']].sample(4)

,sentence,dominant_topic
130,With TD Ameritrade not only can you trade comm...,4
66,To the prior custodian or trustee of the retir...,4
68,Merrill Lynch Pierce Fenner Smith Incorporated...,4
0,Use this form to transfer all or a portion of ...,4


In [0]:
df_sent_topic.loc[df_sent_topic['dominant_topic'] == 12][['sentence', 'dominant_topic']].sample(4)

,sentence,dominant_topic
71,Wealth management is an ongoing process,12
111,For full transfers â indicate all and mark tra...,12
112,For partial transfers â list individual fund n...,12
134,Make informed trades with an incredible depth ...,12


In [0]:
print('There are', len(df_sent_topic.loc[df_sent_topic['dominant_topic'] == 4]), 'sentences that belong to topic 4 and we will remove')
print('There are', len(df_sent_topic.loc[df_sent_topic['dominant_topic'] == 12]), 'sentences that belong to topic 12 and we will remove')

There are 4 sentences that belong to topic 4 and we will remove
There are 5 sentences that belong to topic 12 and we will remove


In [0]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Sentences")
df_topic_distribution.columns = ['Topic #', 'Num Sentences']
df_topic_distribution

,Topic #,Num Sentences
0,8,22
1,16,20
2,36,15
3,18,14
4,0,13
5,24,13
6,26,10
7,17,10
8,29,8
9,34,7


In [0]:
df_sent_topic_clean = df_sent_topic.drop(df_sent_topic[(df_sent_topic.dominant_topic == 4) | (df_sent_topic.dominant_topic == 12)].index)

In [0]:
df_description = df_sent_topic_clean[['sentence','CATEGORY']]
df_description = df_description.groupby('CATEGORY')['sentence'].agg(lambda col: ' '.join(col)).reset_index()

In [0]:
df_description.head()

,CATEGORY,sentence
0,Ameritrade Institutional,A Transfer from an ACAT eligible Brokerage Fir...
1,Merrilllynch_CRA,For best results please complete the informati...
2,Schwab_IRA,Distributions from your IRA are generally subj...


In [0]:
df_description['sentence'][0]

'A Transfer from an ACAT eligible Brokerage Firm Bank Insurance Annuity Co Trust Co or Transfer Agent â Check box for full or partial transfer For partial transfers list descriptions of assets and shares Unless otherwise indicated TD Ameritrade will transfer in full Please note â The ACAT system does not allow for liquidation requests To place trades please contact your current custodian prior to submitting the transfer request Most banks insurance annuity and trust companies require original wet ink signature mailed to TD Ameritrade When transferring from a joint account at a contra firm to an individual account at TD Ameritrade the party losing ownership of the assets must sign section 4 If the party losing ownership does not have an account at TD Ameritrade please attach a notarized letter of authorization signed by the party losing ownership of the assets If the discrepancy is a result of a name change for an account owner please provide a copy of the legal document such as a marri

In [0]:
df_description.set_index('CATEGORY', inplace = True)
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df_description['sentence'])
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [0]:
indices = pd.Series(df_description.index)
def recommendations(name, cosine_similarities = cosine_similarities):
    
    recommended_category = []
  
    idx = indices[indices == name].index[0]

    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending = False)

  
    top_10_indexes = list(score_series.iloc[1:6].index)
    

    for i in top_10_indexes:
        recommended_category.append(list(df_description.index)[i])
        
    return recommended_category

In [0]:
recommendations('Merrilllynch_CRA')

['Ameritrade Institutional', 'Schwab_IRA']